<a href="https://colab.research.google.com/github/laurence-lin/Kaggle_competition/blob/master/Rossman_Store_Sales_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

%tensorflow_version 1.9
import tensorflow as tf
from tensorflow import keras
import x
from sklearn.model_selection import train_test_split

import os
import gc

import warnings
warnings.simplefilter('ignore')
      

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.9`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
from google.colab import files


%cd /root/
!mkdir .kaggle
%cd .kaggle
files.upload()
!kaggle competitions download -c rossmann-store-sales
!unzip test.csv.zip
!unzip train.csv.zip

/root
mkdir: cannot create directory ‘.kaggle’: File exists
/root/.kaggle


Saving kaggle.json to kaggle.json
 75% 5.00M/6.71M [00:00<00:00, 47.8MB/s]
100% 6.71M/6.71M [00:00<00:00, 42.8MB/s]
  0% 0.00/310k [00:00<?, ?B/s]
100% 310k/310k [00:00<00:00, 92.3MB/s]
  0% 0.00/192k [00:00<?, ?B/s]
100% 192k/192k [00:00<00:00, 181MB/s]
  0% 0.00/44.0k [00:00<?, ?B/s]
100% 44.0k/44.0k [00:00<00:00, 43.1MB/s]


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
store = pd.read_csv('store.csv')

NameError: ignored

In [0]:



types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv(f"{data}train.csv", parse_dates=[2], dtype=types)
print(train.info())

'''
train_date = train.Date.unique()
print(train_date)
print('Time scan of dataset', len(train_date))

test_date = test.Date.unique()
print(test_date)
print('Time scan of test data', len(test_date))
'''

Data description:
Totally 942 days for whole dataset, 48 days to predict.

In these 942 days, up to 1115 stores in each day(some store may )

Training dataset: time line 2013.03.01-2015.07.31

Testing dataset: time line 2015-08-01 to 2015-09-17

Stores: 1115 different stores, each store contains 9 features for the store itself

Target: predict daily sales 6 weeks in advance

In [0]:
store = pd.read_csv(data + 'store.csv')

types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv(f"{data}train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv(f'{data}/test.csv', parse_dates = [3], dtype = types)
print(train.info())

# Consider only open store for training.
print(train.shape)
print('Consider only open store for training')
train = train[train.Open == 1]
print(train.shape)
print('Consider only Sales >0, to simplify calculation of RMSPE')
train = train[train.Sales > 0]
print(train.shape)

# Use merge to concat store to train & test set, while 'on' could fit the rows with 'Store'
# merge() method, the two merge dataframe should have same column name and value to fit the merging
train = pd.merge(train, store, on = 'Store')
test = pd.merge(test, store, on = 'Store')
print('Train + Store shape:', train.shape)
print('Test + Store shape:', test.shape)

features = []
print(train.info())
print(test.info())

# fill in missing
obj_feature = train.select_dtypes(exclude = ['int', 'float'])
obj_name = obj_feature.columns.values
print(obj_name)

#for i in range(len(obj_name)):
 #   print(obj_name[i])
  #  print(obj_feature[obj_name[i]].unique())



In [0]:
data_ = train.drop(['Date', 'Open'], axis = 1)
corr = data_.corr()

sns.heatmap(corr)

gc.collect()

In [0]:
corr_sales = corr['Sales'].sort_values(ascending = False)
print(corr_sales)

gc.collect()

In [0]:
train_null = (train.isnull().sum()/len(train)) * 100
print(train_null[train_null > 0])
train_null = train_null[train_null > 0]
train_null.sort_values(ascending = False, inplace = True)
#print('Train missing:\n', train_null)

train_fill = train[train_null.index]
#print(train_fill.info())

# Plot pair plot: While under different categorical features values, 
# the relationship of each features
obj = train.select_dtypes(include = 'object')
obj_feature = obj.columns.values

'''
for feat in obj_feature:
    not_null = [col for col in train.columns if col not in train_fill.columns]
    train_ = train[not_null]
    print(train_.columns)
    plt.figure()
    sns.pairplot(train_, hue = feat)
    plt.show()
'''

gc.collect()

In [0]:
from sklearn.preprocessing import LabelEncoder
print(test.Date.head())
print(train.Date.head())

# Create optimal features by feature engineering
def build_feature(data):
    # Feature engineering: add new features
    features = []
    data.fillna(0, inplace = True)
    
    encode_feature = ['StateHoliday', 'StoreType', 'Assortment']
    encoder = LabelEncoder()
    for i in range(len(encode_feature)):
        data[encode_feature[i]] = encoder.fit_transform(data[encode_feature[i]])
        
    features.extend(['Store', 'StateHoliday', 'StoreType', 'Assortment'])
    
    # Add datetime feature
    date = data.Date
    data['Year'] = data.Date.dt.year
    data['month'] = date.dt.month
    data['day'] = date.dt.day
    data['dayofweek'] = date.dt.dayofweek
    data['weekofyear'] = date.dt.weekofyear
    
    features.extend(['Year', 'month', 'day', 'dayofweek', 'weekofyear'])
    
    # Add in new features for competition of the store: combine the past compitition Open since Year & month
    data['CompetitionOpen'] = 12*(data.Year - data.CompetitionOpenSinceYear) + \
    (data.month - data.CompetitionOpenSinceMonth)
    # unit: past time of month
    # Use month as time interval unit: do not make feature value too large
    data['Promo2Open'] = (data.weekofyear - data.Promo2SinceWeek)/4 + 12*(data.Year - data.Promo2SinceYear) 
    mapping = {'Jan,Apr,Jul,Oct':1, 'Feb,May,Aug,Nov':2}
    data['Promo2Start'] = data.PromoInterval.map(mapping)
    data.fillna(0, inplace = True)
    features.extend(['CompetitionOpen', 'Promo2Open', 'Promo2Start'])
    
    return data, features

'''
def feature_engineer(data):
    data.Year = data.Year - 2000
    scaler = MinMaxScaler()
    data['CompetitionOpen'] = scaler.fit_transform( data['CompetitionOpen'].values.reshape(-1, 1))
    data['Promo2Open'] = scaler.fit_transform(data['Promo2Open'].values.reshape(-1, 1))
    data.drop('Store', axis = 1, inplace = True)
    #print(data.columns)
    scale_feature = ['Year', 'month', 'day', 'weekofyear']
    data['dayofweek'] *= 0.1
    data[scale_feature] *= 0.01

    return data
'''

train, features = build_feature(train)
test, _ = build_feature(test)
print('Training set and Test set feature pre-processed') 

# Get the x & y matrix from specific features
y = train['Sales']
x = train[features]
x_test = test[features]
    

After feature engineering, convert time series data to time pipeline with shape:
[batch size, time step, n_features] 

In [0]:
print('Convert data into time series pipeline: ')
#print(x_train.values.shape)
#print(x_train.iloc[0, :].shape)
gc.collect()
# Create time pipeline for LSTM network
def create_pipeline(data, target, time_step = 3):
    # Use t ~ (t + t_delta) features to predict Sales on time (t + t_delta)
    # data: [batch size, features]
    # target: [batch_size], output target of each data sample
    # time_step: number of past time step to predict future
    # return: time pipeline of data size [batch size_, time_step, features] and target
    pipeline = []
    for t in range(len(data) - time_step + 1):
        #print(data[0].shape)
        pipeline.append(np.array(data[t:(t + time_step)]))
    
    target = target[(time_step - 1):]
    pipeline = np.array(pipeline)
   
    return pipeline, target

time_step = 10
x, y = create_pipeline(x, y, time_step)
print('Training set converted to time pipeline')
print('Train data after convert: ', x.shape)
print('Train target after convert: ', y.shape)

gc.collect()
    

After feature engineering and time pipeline created, we could start build our model

In [0]:
# Define the prediction model: LSTM
def create_model(time_step, input_features):
    '''
    Time series forecasting model
    '''
    model = keras.models.Sequential()
    # input shape to LSTM layer: [batch size, time step, features]
    model.add(keras.layers.LSTM(units = 20, input_shape = (time_step, input_features))) 
    model.add(keras.layers.Dense(1, activation = tf.nn.relu))
    
    return model

seed = 9
x_train, x_valid, y_train , y_valid = train_test_split(x, y, test_size = 0.2, 
                                                       random_state = seed)

input_features = x_train.shape[-1]
'''
model = create_model(time_step, input_features)
model.compile(optimizer = tf.train.AdamOptimizer(1e-3),
             loss = 'mse',
             metrics = ['mse'])
'''
model = keras.models.load_model('../input/lstm-model/lstm_model.h5')
model.compile(optimizer = tf.train.AdamOptimizer(1e-3),
             loss = 'mse',
             metrics = ['mse'])
# Save checkpoint model during training
filepath = 'lstm_model.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                monitor = 'val_loss', 
                                                save_best_only = True,
                                                mode = 'min', # while val_loss is minimum
                                                period = 5,
                                                verbose = 1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                mode = 'min',
                                                patience = 5)

history = model.fit(x_train, y_train, epochs = 100,
            shuffle = True,
            validation_data = (x_valid, y_valid),
            callbacks = [checkpoint, earlystopping])

model.save('lstm_model.h5')

In [0]:
# Define parameters of xgboost
param = {
    'booster':'gbtree',
    'eta': 0.3,  # learning rate
    'gamma':0,  # minimal loss to split leaf
    'max_depth':10,
    'predictor':'gpu_predictor'
        }

# Split train and valid data
seed = 9
x_train, x_valid = train_test_split(train, test_size = 0.2, random_state = seed)
# input of xgboost should be dmatrix


print(features)

dtrain = xgb.DMatrix(x_train[features], label = x_train.Sales)
dvalid = xgb.DMatrix(x_valid[features], label = x_valid.Sales)
print(x_train.columns)

num_round = 300
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
# Training
#regressor = xgb.train(param, dtrain, num_round, evals = watchlist, early_stopping_rounds = 100)

In [0]:
y_pred = model.predict(x_valid)


y_pred = np.squeeze(y_pred)
rmse = np.sqrt(sum((np.squeeze(y_pred) - y_true)**2))/len(y_true)
print('RMSE:', rmse)

y_pred = y_pred.ravel()
y_true = y_true.ravel()
print(y_pred[0:20])
plt.plot(y_pred[0:20], color = 'red', label = 'predicted value')
plt.plot(y_true[0:20], color = 'blue', label = 'true value')
plt.legend()
plt.show()



In [0]:
# Calculate validate accuracy and show the result
print('Validating')

y_pred = regressor.predict(dvalid)
rmse = np.sqrt(sum((y_pred - x_valid.Sales)**2))/len(y_pred)
print('RMSE:', rmse)

print(x_valid.Sales.values.shape)

plt.plot(x_valid.Sales.values[0:10000], color = 'red')
plt.plot(y_pred[0:10000], color = 'blue')
plt.show()


In [0]:
#test = xgb.DMatrix(test[features])
prediction = regressor.predict(test)
submit = '/kaggle/input/sample_submission.csv'
submit = pd.read_csv(submit)
print(submit.head())
submit.Sales = prediction
submit.to_csv('submission.csv')
print(submit.head())

In [0]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# create a link to download the dataframe
create_download_link(submit)

